In [50]:
!pip install unsloth vllm
!pip install --upgrade pillow
!pip install llama-cpp-python
!pip install langchain-community
!pip install wandb


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 65.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import torch
import re
import pandas as pd
from unsloth import FastLanguageModel, PatchFastRL, is_bfloat16_supported
from datasets import load_dataset, Dataset
from difflib import SequenceMatcher

PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = 512,
    max_lora_rank = 64,
    load_in_4bit = True,
    fast_inference = True,
    gpu_memory_utilization = 0.8)

model = FastLanguageModel.get_peft_model(
    model,
    r = 64,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = 64,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407)


INFO 03-05 14:54:26 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA L4. Max memory: 21.951 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit with actual GPU utilization = 79.32%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 21.95 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 224.
Unsloth: vLLM's KV Cache can use up to 11.08 GB. Also swap space = 3 GB.
INFO 03-05 14:54:37 config.py:549] This model supports multiple tasks: {'score', 'reward', 'embed', 'generate', 'classify'}.

[W305 14:54:39.987754988 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 03-05 14:54:39 loader.py:1089] Loading weights with BitsAndBytes quantization.  May take a while ...
INFO 03-05 14:54:40 weight_utils.py:254] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 03-05 14:55:10 model_runner.py:1115] Loading model weights took 5.5976 GB
INFO 03-05 14:55:10 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 03-05 14:55:13 worker.py:267] Memory profiling takes 2.81 seconds
INFO 03-05 14:55:13 worker.py:267] the current vLLM instance can use total_gpu_memory (21.95GiB) x gpu_memory_utilization (0.79) = 17.41GiB
INFO 03-05 14:55:13 worker.py:267] model weights take 5.60GiB; non_torch_memory takes 0.04GiB; PyTorch activation peak memory takes 1.04GiB; the rest of the memory reserved for KV Cache is 10.73GiB.
INFO 03-05 14:55:13 executor_base.py:111] # cuda blocks: 5494, # CPU blocks: 1536
INFO 03-05 14:55:13 executor_base.py:116] Maximum concurrency for 512 tokens per request: 171.69x
INFO 03-05 14:55:15 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error o

Capturing CUDA graph shapes: 100%|██████████| 31/31 [00:33<00:00,  1.08s/it]

INFO 03-05 14:55:49 model_runner.py:1562] Graph capturing finished in 34 secs, took 0.76 GiB
INFO 03-05 14:55:49 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 38.98 seconds



Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
cs_qa = load_dataset('tau/commonsense_qa')
cs_qa = pd.DataFrame(cs_qa['train'])

# Save the train split (or any available split) to a CSV file
cs_qa.to_csv("data/commonsense/commonsense_qa.csv", index=False)

In [3]:
piqa = load_dataset('ybisk/piqa')
piqa = pd.DataFrame(piqa['train'])

piqa.to_csv("data/commonsense/piqa.csv", index=False)

In [4]:
piqa = pd.read_csv("data/commonsense/piqa.csv")
piqa.head()

,goal,sol1,sol2,label
0,"When boiling butter, when it's ready, you can",Pour it onto a plate,Pour it into a jar,1.0
1,"To permanently attach metal legs to a chair, y...",Weld the metal together to get it to stay firm...,Nail the metal together to get it to stay firm...,0.0
2,how do you indent something?,leave a space before starting the writing,press the spacebar,0.0
3,how do you shake something?,move it up and down and side to side quickly.,stir it very quickly.,0.0
4,Clean tires,"Pour water, cape off caked on dirt. Use speed...","Pour water, scrape off caked on dirt. Use a st...",1.0


In [5]:
piqa[piqa['goal'].str.contains("how")]

,goal,sol1,sol2,label
2,how do you indent something?,leave a space before starting the writing,press the spacebar,0.0
3,how do you shake something?,move it up and down and side to side quickly.,stir it very quickly.,0.0
5,how do you taste something?,smell it enough to taste it.,place it in your mouth to taste.,1.0
9,how do you stab something?,stick a sharp object through it.,pin it with a sharp object.,0.0
12,how do you open a capri-sun,"open the straw attached to the juice, and then...",cut the top off with scissors,0.0
...,...,...,...,...
16101,how do you use syrup?,"pour it over pancakes, waffles or french toast.","pour it into chocolate milk, or strawberry milk.",0.0
16103,how to clean stove burners,"pop them off, and let soak for six weeks in a ...","pop them off, and let soak overnight in a cont...",1.0
16104,how do you get a spare of something?,get four of the same thing,get two of the same thing.,1.0
16108,how to make juice grooves on a wood board,use a cnc router,use a pizza wheel,0.0


In [41]:
# System prompt - adapted from R1 paper
SYSTEM_PROMPT = """
A conversation between user and assistant. user asks a question and gives options A and B. 
Assistant thinks about the reasoning process and provides the user with A or B.
Responses should be in this format:
<reason>
...
</reason>
<answer>
...
</answer>
"""

def extract_xml_tag(text: str, tag: str) -> str:
    answer = text.split(f"<{tag}>")[-1]
    answer = answer.split(f"</{tag}>")[0]
    return answer.strip()

# uncomment middle messages for 1-shot prompting
def format_data() -> Dataset:
    # Load CSVs
    piqa = pd.read_csv("data/commonsense/piqa.csv")
    piqa = piqa[piqa['goal'].str.contains('how')]

    # Convert to structured format
    records = piqa.apply(lambda row: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': f"{row['goal']} \nA: {row['sol1']} B: {row['sol2']}"}
        ],
        'answer': "A" if row['label'] == 0 else "B"
    }, axis=1).to_list()
    
    return Dataset.from_list(records)

dataset=format_data()

In [42]:
def filter_long_prompts(example, tokenizer, max_length=512):
    full_prompt = "\n".join(
        [f"{turn['role']}: {turn['content']}" for turn in example['prompt']]
    )
    tokens = tokenizer(full_prompt, add_special_tokens=False).input_ids
    return len(tokens) <= max_length

# Use filter with datasets.Dataset
dataset = dataset.filter(lambda x: filter_long_prompts(x, tokenizer, max_length=512))


Filter:   0%|          | 0/2240 [00:00<?, ? examples/s]

## Reward Functions

### Correct Answer
Rewards if the answer and preferred answer are similar.

In [10]:
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'].strip().lower() for completion in completions]
    extracted_responses = [extract_xml_tag(r, "answer").strip().lower() for r in responses]
    correct_answers = [a.strip().lower() for a in answer]

    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, correct_answers)]

### Strict Correct XML Format
Rewards a perfectly formatted response with proper XML tag order and structure

In [11]:
def strict_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"^<reason>\n.*?\n</reason>\n<answer>\n.*?\n</answer>$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) and r.endswith("</answer>") and r.startswith("<clue>") for r in responses]
    return [0.75 if match else 0.0 for match in matches]

### Soft Correct XML Format
Rewards a response as long as the XML is parsable. Does not whitespace.

In [12]:
def soft_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"^<reason>\s*.*?\s*</reason>\s*<answer>\s*.*?\s*</answer>$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

### XML Count
Rewards if the correct number of XML tags are present in any order

In [13]:
def count_xml(text) -> float:
    count = 0.0
    if text.count("<reason>\n") == 1:
        count += 0.125
    if text.count("\n</reason>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
    if text.count("\n</answer>") == 1:
        count += 0.125
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

### Answer Brevity
Rewards single letter answers

In [14]:
def short_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'].strip() for completion in completions]
    extracted_responses = [extract_xml_tag(r, "answer").strip() for r in responses]
    
    def shortness_score(r):
        if len(r) == 1:
            return 1.0
        return 0.0
    
    return [shortness_score(r) for r in extracted_responses]

### Train Model

In [15]:
import wandb

# Initialize WandB
wandb.init(project="piqa-llm-training", name="grpo_run_1", config={
    "learning_rate": 5e-6,
    "num_train_epochs": 10,
    "batch_size": 1,
    "max_steps": 50000,
    "gradient_accumulation_steps": 1,
})

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: peytonnash (peytonnash-uc) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [20]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True,
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, 
    num_generations = 8, 
    max_prompt_length = 500,
    max_completion_length = 200,
    num_train_epochs = 10, 
    max_steps = 10000,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "wandb", 
    output_dir = "outputs",
)

# Log training arguments to WandB
wandb.config.update(vars(training_args))

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        short_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,235 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 1
\        /    Total batch size = 8 | Total steps = 10,000
 "-____-"     Number of trainable parameters = 167,772,160


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / short_reward_func,rewards / correctness_reward_func
1,0.000000,0.437500,0.538682,161.375000,0.000864,0.062500,0.000000,0.000000,0.375000,0.000000
2,0.000000,0.218750,0.247758,87.125000,0.000980,0.156250,0.062500,0.000000,0.000000,0.000000
3,0.000000,0.609375,1.148441,149.250000,0.000812,0.109375,0.000000,0.000000,0.250000,0.250000
4,0.000100,0.984375,1.326982,82.625000,0.001781,0.234375,0.000000,0.000000,0.250000,0.500000
5,0.000000,0.890625,1.072042,122.125000,0.001184,0.140625,0.000000,0.000000,0.500000,0.250000
6,0.000000,0.234375,0.169525,100.375000,0.000961,0.234375,0.000000,0.000000,0.000000,0.000000
7,0.000000,1.140625,1.566157,152.000000,0.001057,0.203125,0.062500,0.000000,0.375000,0.500000
8,0.000000,0.703125,1.113909,142.375000,0.001038,0.203125,0.000000,0.000000,0.250000,0.250000
9,0.000000,1.359375,1.574685,92.500000,0.001166,0.234375,0.000000,0.000000,0.375000,0.750000
10,0.000000,0.750000,0.578638,145.750000,0.000868,0.062500,0.062500,0.000000,0.625000,0.000000


In [38]:
def check_prompt_lengths(dataset, tokenizer, max_length=512):
    lengths = []
    for example in dataset:
        # Combine system and user messages into one string (if you store them separately)
        full_prompt = "\n".join(
            [f"{turn['role']}: {turn['content']}" for turn in example['prompt']]
        )
        tokens = tokenizer(full_prompt, add_special_tokens=False).input_ids
        lengths.append(len(tokens))
    
    over_limit = [l for l in lengths if l > max_length]
    print(f"Checked {len(lengths)} prompts")
    print(f"Max length: {max(lengths)} tokens")
    print(f"{len(over_limit)} prompts exceed {max_length} tokens")


In [43]:
check_prompt_lengths(dataset, tokenizer, max_length=512)

Checked 2235 prompts
Max length: 478 tokens
0 prompts exceed 512 tokens
